# The Rosenbrock’s Function
Let's optimize the Rosenbrock’s function

$$
f_{\text{rosenbrock}}(\mathbf{x})==\sum_{i=1}^{D-1}\left[100\left(\mathbf{x}_i^2-\mathbf{x}_{i+1}\right)^2+\left(\mathbf{x}_i-1\right)^2\right]
$$

whose global optimum is $f_{\text{rosenbrock}}(\mathbf{x}^*=(1,1,\ldots,1)^T) = 1$.

The function can be seen as assembled by $D-1$ elements, namely

$$
f_{\text{rosenbrock}}(\mathbf{x})=\sum_{i=1}^{D-1} f_i(\mathbf{x}),
$$

where each $f_i(\mathbf{x}) = 100\left(\mathbf{x}_i^2-\mathbf{x}_{i+1}\right)^2+\left(\mathbf{x}_i-1\right)^2$ depends on the $i$-th and $(i+1)$-th variables of $\mathbf{x}$.

In [1]:
import matplotlib.pyplot as plt
from upoqa.problems import PSProblem
import numpy as np

# Define the element function
def rosenbrock_ele(z):    # input is z = [x[i-1], x[i]] for each i-th element
    return 100.0 * (z[0] ** 2 - z[1]) ** 2 + (z[0] - 1) ** 2

# there are 99 elements
D = 100

# every element is the same as rosenbrock_ele
elements = dict([(f"ele {i}", rosenbrock_ele) for i in range(1, D)])

# the i-th element depends on x[i-1] and x[i] 
coords = dict([(f"ele {i}", [i - 1, i]) for i in range(1, D)])

# create the problem
prob = PSProblem(elements, coords)
prob.update_meta_info({'name': 'Rosenbrock'})
print(prob)

# let's start from zero
x0 = np.zeros(prob.dim)

The Rosenbrock Problem with 99 Elements and 100 Dimensions.


Let's try directly using `scipy.optimize.minimize` first, which uses the BFGS algorithm by default.
It costs over 60000 evaluations to converge.

In [26]:
from scipy.optimize import minimize as scipy_minimize
scipy_res = scipy_minimize(prob.fun_eval, x0)
print(scipy_res)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 7.245474453262833e-11
        x: [ 1.000e+00  1.000e+00 ...  1.000e+00  1.000e+00]
      nit: 485
      jac: [ 2.020e-07 -6.941e-07 ... -5.211e-07 -2.369e-07]
 hess_inv: [[ 2.447e-03  1.207e-03 ... -1.041e-04 -2.416e-04]
            [ 1.207e-03  2.491e-03 ... -4.604e-05 -1.080e-04]
            ...
            [-1.041e-04 -4.604e-05 ...  3.435e-01  6.860e-01]
            [-2.416e-04 -1.080e-04 ...  6.860e-01  1.375e+00]]
     nfev: 64640
     njev: 640


Then `upoqa`, which uses less than 1000 evaluations to converge.

In [2]:
import upoqa
res = upoqa.minimize(elements, x0 = x0, coords = coords)

Initializing surrogate models: 100%|██████████| 99/99 [00:00<00:00, 2213.93it/s, element=ele 99]
Searching for better x0... done, point num = 5000, best fun = 99.0


 Run  Iter     Obj       Grad      Delta    Resolution  Evals 
  1     3    9.90e+01  5.00e+01  1.26e-01    1.00e-01     7   
  1     4    9.79e+01  1.21e+01  1.26e-01    1.00e-01     9   
  1     8    9.79e+01  5.16e+00  2.56e-02    1.00e-02    12   
  1     9    9.78e+01  2.30e+00  2.58e-02    1.00e-02    14   
  1    10    9.77e+01  1.24e+00  2.64e-02    1.00e-02    15   
  1    11    9.77e+01  1.25e+00  1.84e-02    1.00e-02    16   
  1    12    9.76e+01  2.64e+00  1.91e-02    1.00e-02    18   
  1    13    9.75e+01  3.30e+00  1.93e-02    1.00e-02    19   
  1    14    9.75e+01  2.26e+01  1.34e-02    1.00e-02    20   
  1    15    9.73e+01  6.32e+00  1.43e-02    1.00e-02    22   
  1    16    9.70e+01  3.94e+00  1.52e-02    1.00e-02    23   
  1    17    9.68e+01  1.47e+01  1.72e-02    1.00e-02    24   
  1    18    9.68e+01  8.49e+00  1.23e-02    1.00e-02    25   
  1    19    9.67e+01  6.40e+00  1.26e-02    1.00e-02    26   
  1    20    9.64e+01  3.94e+00  1.30e-02    1.00e-02  

In [6]:
print(f"Objective function value at the solution = {res.fun}")
print(f"Maximum number of element function evaluations = {res.max_nfev}")

Objective function value at the solution = 2.7210098453897897e-10
Maximum number of element function evaluations = 880
